### Notebook to format dates in the point 46, 47, 48 csv files, to avoid dates being in different formats (all in classic format dd/mm/yyyy)

In [23]:
import os 
import pandas as pd
import numpy as np

path = "C:/Users/alexa/2A/MOLONARI1D/dataAnalysis/data_traite"
os.listdir(path)[2:]

L=['point46_pression_traité.csv', 'point46_temperature_traité.csv', 'point47_pression_traité.csv', 'point47_temperature_traité.csv', 'point48_pression_traité.csv', 'point48_temperature_traité.csv']

for file in L:
    k=0
    with open(path+'/'+file, 'r') as f:
        data = f.readlines()
        f.close()
    with open('new_' + file, 'w') as new_file:
        new_file.write(data[0])
        new_file.write(data[1])
        for i, line in enumerate(data[2:]):
            if k==0:
                if int(line[0:2]) == int(data[i+1][0:2]) or int(line[0:2]) == int(data[i+1][0:2]) + 1:
                    new_file.write(line)
                else:
                    k=1
                    nd = line[3:5]
                    nm = line[0:2]
                    line = nd + line[2] + nm + line[5:]
                    new_file.write(line)
                    data[i+2] = line
            elif k==1:
                nd = line[3:5]
                nm = line[0:2]
                if int(line[3:5])==int(data[i+1][0:2]) or int(line[3:5])==int(data[i+1][0:2])+1:
                    line = nd + line[2] + nm + line[5:]
                    new_file.write(line)
                    data[i+2] = line
                else:
                    k=0
                    new_file.write(line)

In [24]:
def convertDates(df: pd.DataFrame):
    """
    Convert dates from a list of strings by testing several different input formats
    Try all date formats already encountered in data points
    If none of them is OK, try the generic way (None)
    If the generic way doesn't work, this method fails
    (in that case, you should add the new format to the list)
    
    This function works directly on the giving Pandas dataframe (in place)
    This function assumes that the first column of the given Pandas dataframe
    contains the dates as characters string type
    
    For datetime conversion performance, see:
    See https://stackoverflow.com/questions/40881876/python-pandas-convert-datetime-to-timestamp-effectively-through-dt-accessor
    """
    formats = ("%m/%d/%y %H:%M:%S", "%m/%d/%y %I:%M:%S %p",
               "%d/%m/%y %H:%M",    "%d/%m/%y %I:%M %p",
               "%m/%d/%Y %H:%M:%S", "%m/%d/%Y %I:%M:%S %p", 
               "%d/%m/%Y %H:%M",    "%d/%m/%Y %I:%M %p",
               "%y/%m/%d %H:%M:%S", "%y/%m/%d %I:%M:%S %p", 
               "%y/%m/%d %H:%M",    "%y/%m/%d %I:%M %p",
               "%Y/%m/%d %H:%M:%S", "%Y/%m/%d %I:%M:%S %p", 
               "%Y/%m/%d %H:%M",    "%Y/%m/%d %I:%M %p",
               None)
    times = df[df.columns[0]]
    for f in formats:
        try:
            # Convert strings to datetime objects
            new_times = pd.to_datetime(times, format=f)
            # Convert datetime series to numpy array of integers (timestamps)
            new_ts = new_times.values.astype(np.int64)
            # If times are not ordered, this is not the appropriate format
            test = np.sort(new_ts) - new_ts
            if np.sum(abs(test)) != 0 :
                #print("Order is not the same")
                raise ValueError()
            # Else, the conversion is a success
            #print("Found format ", f)
            df[df.columns[0]] = new_times
            return
        
        except ValueError:
            #print("Format ", f, " not valid")
            continue
    
    # None of the known format are valid
    raise ValueError("Cannot convert dates: No known formats match your data!")


In [25]:
#Testing if convertDates works

df = pd.read_csv('new_point46_pression_traité.csv')
convertDates(df)

#Working 

C:\Users\alexa\AppData\Local\Temp\ipykernel_7720\1511527849.py:29: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  new_times = pd.to_datetime(times, format=f)
